In [ ]:
#Importing necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/AI Society Fall23/Domo/preprocessed_data.csv')

In [ ]:
data.shape

(376, 8)

In [ ]:
df  = data[['event_name', 'datetime', 'description', 'host_org', 'event_perks', 'location']]

In [ ]:
#Fill the na values with unknown in host_org, event_perks features in df dataframe

df['host_org'].fillna('unknown', inplace=True)
df['event_perks'].fillna('unknown', inplace=True)

<ipython-input-39-2711c94ff00b>:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['host_org'].fillna('unknown', inplace=True)
<ipython-input-39-2711c94ff00b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['host_org'].fillna('unknown', inplace=True)
<ipython-input-39-2711c94ff00b>:4: FutureWarning: A value is trying to be set on a copy o

In [ ]:
data['host_org'] = data['host_org'].fillna('')
data.event_perks = data.event_perks.fillna('')
data.categories = data.categories.fillna('')

In [ ]:
data['information'] = data['description'] + ' ' + data['host_org'] + ' ' + data['event_perks'] + ' ' + data['categories'] + ' ' + data['location']

In [ ]:
import re
import string
!pip install contractions
import contractions
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
def preprocess_text(data):

    # Convert to lowercase
    if isinstance(data, pd.Series):
        data = data.astype(str).apply(lambda x: x.lower())
    else:
        data = data.lower()

    # Remove HTML tags
    data = re.sub(r'<.*?>', '', data)

    # Remove URLs
    data = re.sub(r'https?://[^\s]+', '', data)

    # Remove mentions
    data = re.sub(r'@\w+', '', data)

    # Remove hashtags
    data = re.sub(r'#\w+', '', data)

    # Remove special characters and punctuation
    data = re.sub(r'[^a-zA-Z0-9\s]', '', data)

    # Remove punctuation
    data = data.translate(str.maketrans('', '', string.punctuation))

    # Remove digits
    data = ''.join([i for i in data if not i.isdigit()])

    # Remove extra whitespace
    data = re.sub(r'\s+', ' ', data.strip())

    # Performing contractions
    data = contractions.fix(data)

    # Remove stop words using NLTK
    stop = nltk.corpus.stopwords.words('english')
    data = ' '.join([x for x in data.split() if x not in (stop)])

    return data

In [ ]:
data.information = data.information.apply(preprocess_text)

In [ ]:
def recommned(user_input):

  # Preprocess user input
  user_data = preprocess_text(user_input)

  # Combining ifidf from data.
  from sklearn.feature_extraction.text import TfidfVectorizer
  vectorizer = TfidfVectorizer(max_features=10000)

  # Fit the vectorizer on the combined text data
  vectorizer.fit(data.information)

  # Transform the user input
  user_ifidf = vectorizer.transform([user_data])

  # Transform the text data
  df_tfidf = vectorizer.transform(data.information)

  # Calculate cosine similarity
  from sklearn.metrics.pairwise import cosine_similarity
  cosine_sim = cosine_similarity(user_ifidf, df_tfidf)

  # Get top 5 recommendations
  top_indices = cosine_sim.argsort().flatten()[-5:][::-1]
  recommendations = df.iloc[top_indices]
  return recommendations

# Finally finished!

In [ ]:
recommned('I like music, free food, and networking')

,event_name,datetime,description,host_org,event_perks,location
31,Art After Dark,2024-08-17 19:30:00,come student union enjoy evening music crafts ...,polytechnic,food,student union
292,"School of Music, Dance and Theatre TA Training...",2024-08-19 15:30:00,welcome new teaching assistants school music d...,host organizations school music dance theatre ...,unknown,katzin concert hall e gammage dr tempe
137,Glo BINGO Hosted by the College of Integrative...,2024-09-06 18:00:00,prepare unforgettable evening packed bright li...,host organizations college integrative science...,food stuff,polytechnic cooley
261,Popsicles and Yard Games,2024-08-22 20:00:00,come eat popsicles play yard games us school m...,young life,unknown,school music courtyard gammage gammage pkwy tempe
185,Join Us! 1st Fall Semester Meeting!,2024-09-04 16:30:00,join us small networking event club members we...,shrm,unknown,online
